### **Semantic Alignment(의미적 일치성) 설계 가이드**

**1. 측정 원리**: 

Input(입력 의도)과 Output(생성된 키워드) 사이의 거리를 측정.

- Input Text (의도): "A quiet and instrumental music playlist for studying in a library." (Taxonomy를 자연어 문장으로 변환)
- Output Text (결과): "Lo-fi beats for focus", "Calm piano" (GPT가 뱉은 검색어 리스트)

**목표** : 이 두 텍스트가 얼마나 의미적으로 가까운지 점수(0.0 ~ 1.0)로 환산하는 것
 
---

**2. 방법 A([1] 정량적 방법(임베딩)): 임베딩 코사인 유사도 (Quantitative)**:

텍스트를 숫자 벡터로 바꿔서 수학적으로 거리를 계산. 빠르고 비용이 저렴함.

- **도구**: OpenAI text-embedding-3-small 또는 무료 오픈소스 all-MiniLM-L6-v2 (Sentence-Transformers)
- **공식**: $\text{Similarity} = \cos(\theta) = \frac{A \cdot B}{\|A\| \|B\|}$

In [1]:
from sklearn.metrics.pairwise import cosine_similarity
# 만약 OpenAI API 쓴다면 embeddings utils 사용, 여기선 로컬 라이브러리 예시
from sentence_transformers import SentenceTransformer
import numpy as np

# 1. 모델 로드 (가볍고 빠른 모델)
model = SentenceTransformer('all-MiniLM-L6-v2')

def calculate_semantic_alignment(taxonomy_input, generated_keywords):
    """
    taxonomy_input: dict (예: {'location': 'library', 'goal': 'focus'})
    generated_keywords: list (예: ['lo-fi beats', 'calm piano'])
    """
    
    # 2. 비교할 기준 문장(Anchor) 만들기
    # Taxonomy를 풀어서 '의도'를 담은 문장으로 변환
    anchor_text = f"Music for {taxonomy_input['goal']} in a {taxonomy_input['location']} environment with {taxonomy_input['decibel']} noise level."
    
    # 3. 생성된 키워드들을 하나의 문장으로 합치기
    target_text = ", ".join(generated_keywords)
    
    # 4. 벡터화 (Embedding)
    embeddings = model.encode([anchor_text, target_text])
    
    # 5. 코사인 유사도 계산
    score = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]
    
    return score # 0.0 ~ 1.0 사이 값 (1.0에 가까울수록 일치)

# --- 테스트 ---
input_ctx = {'location': 'library', 'goal': 'focus', 'decibel': 'silent'}
output_keys = ['heavy metal rock', 'loud drums'] # ❌ 나쁜 예시

score = calculate_semantic_alignment(input_ctx, output_keys)
print(f"일치도 점수: {score:.4f}") 
# 예상 결과: 0.1~0.2 (매우 낮음)

c:\Users\seonu\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


일치도 점수: 0.4353


---

**3. 방법 B([2] 정성적 방법(LLM Judge)): LLM-as-a-Judge (Qualitative)**

GPT-4o나 저렴한 GPT-3.5를 '채점관(Judge)'으로 쓰는 방식. 수치로 잡기 힘든 미묘한 뉘앙스를 평가할 때 좋음.

**방식**: 채점용 프롬프트를 따로 짜서, GPT에게 "이 추천이 말이 되니?"라고 물어보고 1~5점 점수를 받음.

**장점**: "왜 점수가 낮은지" 이유를 들을 수 있음.

**[채점용 프롬프트 가이드]**

''' # System Prompt
너는 음악 추천 시스템의 품질 관리자(QA Engineer)이다.
입력된 '사용자 상황'과 추천된 '검색 키워드'를 비교하여 적합성을 평가하라.

''' # 평가 기준 (Score 1-5)
5점: 상황과 목표에 완벽하게 부합하며, 음악적 분위기가 정확함.
3점: 대체로 무난하지만, 일부 키워드가 상황에 맞지 않음 (예: 독서실인데 약간 시끄러운 곡 포함).
1점: 상황과 정반대되거나 유해한 추천 (예: 수면 목표인데 헤비메탈 추천).

''' # Input
- Context: {location}, {decibel}, {goal}
- Recommended Keywords: {keywords}

''' # Output Format (JSON)
{
  "score": (int),
  "reason": "한 줄 평가"
} 

---

**4. 최종평가**

단순히 두 점수를 더해서 평균 내는 것보다, **정량적 지표(임베딩)**를 1차 필터로 쓰고, **정성적 지표(LLM Judge)**를 2차 검증관으로 쓰는 구조 사용.

**1. 평가 파이프라인 (Workflow)**

- Step 1 (Speed Check): 모든 결과에 대해 **정량적 평가(임베딩 유사도)**를 돌리기.(싸고 빠름) -> SentenceTransformer

- Step 2 (Quality Check): 임베딩 점수가 **'애매한 구간'**이면 랜덤 샘플링(10%)에 대해 **정성적 평가(LLM Judge)**를 수행. -> OpenAI API

- Step 3 (Human Check): LLM Judge가 점수를 낮게 준 케이스만 직접 눈으로 확인하고 프롬프트를 수정.

**2. 최종 점수 산출 공식 (Scoring Formula)**:

Final Quality Score = ($W_1$ × Embedding Score) + ($W_2$ × LLM Judge Score)

- 개발단계와 운영단계의 가중치가 달라야함

A. 개발 및 테스트 단계 

- 이때는 정확도가 중요함-> LLM의 판단을 더 신뢰.

- 가중치: 임베딩(30%) + LLM Judge(70%)

- 이유: 임베딩은 '뉘앙스'를 못 읽음. 예를 들어 Silent 환경에서 Soft Rock이 나왔을 때, 임베딩은 "Rock이니까 틀렸어"라고 할 수 있지만, LLM은 "Soft니까 괜찮아"라고 판단할 수 있음.

B. 대시보드 및 운영 단계

- 데이터가 수천 건 쌓이면 LLM 비용이 비싸지니까 임베딩 비중을 높임.

- 가중치: 임베딩(80%) + LLM Judge(20% - 랜덤 샘플링)


---

### **->Semantic Alignment KPI 정의서**

| 구분 | 지표명 (Metric Name) | 목표치 (Target) | 설명 및 의도 |
|------|-----------------------|------------------|---------------|
| Main KPI | Avg. LLM Judge Score | 4.5 / 5.0 이상 | 전반적인 추천 품질의 평균 수준. “대부분의 추천이 훌륭한가?”를 측정. |
| Sub KPI 1 | Fatal Error Rate (치명적 오류율) | 0% (Zero) | LLM 점수가 1점(최악)인 케이스의 비율. “사용자가 불쾌감을 느낄만한 엉뚱한 추천이 단 하나도 없는가?” (신뢰성 방어) |
| Sub KPI 2 | Embedding Similarity | 0.75 이상 | 정량적 자동화 지표. 대량의 데이터를 빠르게 검증할 때의 최소 합격 기준선. |


---

11/20 지금 당장은 Ground Truth(실제 데이터값)이 없기 때문에 평가셋(내가 생각하는 정답으로 대체)을 만들어서 성능 평가 자동화 기능 만들기

**Soundscape 평가 데이터셋 v0.1 (20 Cases)**

evaluation_set_v1.csv

**구조 설명**:

- Input: Location, Decibel, Goal, User Preference (Genre/Artist)

- Ground Truth (정답지): Expected Keywords / Vibe (기대 결과), 우리가 기대하는 키워드나 분위기 (LLM Judge가 채점할 때 참고할 기준)

- Type: 테스트 목적 (일반 추천 vs 충돌 해결 vs 감정 처리)

In [ ]:
import pandas as pd
from openai import OpenAI
import json
import time
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# ===========================================================
# 1. 설정 (Configuration)
# ===========================================================
# 🔑 여기에 OpenAI API Key를 입력하세요!
API_KEY = "YOUR_KEY_HERE"

client = OpenAI(api_key=API_KEY)
INPUT_FILE = "evaluation_set_v1.csv"          # 아까 저장한 평가셋 파일명
OUTPUT_FILE = "evaluation_result_final.xlsx"  # 결과가 저장될 엑셀 파일명

#  [무료] 내 컴퓨터에서 돌아가는 임베딩 모델 로드 (최초 1회 다운로드)
print("📥 로컬 임베딩 모델(무료) 로딩 중... (잠시만 기다려주세요)")
local_model = SentenceTransformer('all-MiniLM-L6-v2')
print("✅ 모델 로딩 완료!")

# ===========================================================
# 2. 프롬프트 설계 (Prompt Engineering)
# ===========================================================

# [DJ 역할] 백엔드에 전달한 것과 동일한 추천 엔진 로직
SYSTEM_PROMPT_DJ = """
# Role
당신은 사용자의 실시간 환경(Context)과 음악 취향(Preference)을 분석하여, Spotify 검색 키워드를 생성하는 AI DJ 'Soundscape'입니다.

# Recommendation Logic
1. **Context First**: 취향보다 현재 '목표(Goal)'와 '소음(Decibel)'이 우선입니다.
2. **Conflict Resolution**: 상황과 취향 충돌 시, 취향의 하위/유사 장르 중 상황에 맞는 것을 선택합니다.
3. **Goal-Specific Mood**:
   - anger: High Energy + Low Valence
   - consolation: Low Energy + High Acousticness
   - sleep: Low BPM, Low Energy

# Output Format (JSON Only)
반드시 아래 JSON 구조를 정확히 지켜주세요. 다른 말은 하지 마세요.

{
  "recommendation_meta": {
    "reasoning": "한글 1문장. 추천 근거",
    "primary_tag": "mood_tag"
  },
  "spotify_search_params": {
    "search_queries": [
      "English Query 1",
      "English Query 2",
      "English Query 3"
    ],
    "target_audio_features": {
       "min_tempo": (int),
      "max_tempo": (int),
      "target_energy": (float 0.0~1.0),
      "target_instrumentalness": (float 0.0~1.0),
      "target_valence": (float 0.0~1.0),
      "target_acousticness": (float 0.0~1.0)
    }
  }
}
"""

# [Judge 역할] 추천 품질을 채점하는 평가관 로직
SYSTEM_PROMPT_JUDGE = """
# Role
당신은 음악 추천 품질을 평가하는 QA Judge입니다.
'사용자 상황(Context)'과 '정답 기준(Expected)', 그리고 'AI 추천 결과(Output)'를 비교하여 점수를 매깁니다.

# Scoring Criteria (1-5)
- 5점 (Perfect): 상황/목표에 완벽히 부합하며, 정답 키워드의 뉘앙스를 정확히 캐치함.
- 3점 (Fair): 나쁘지 않으나, 디테일이 부족하거나 베스트는 아님.
- 1점 (Fatal Error): 상황과 정반대되거나(예: 수면에 락), 사용자의 핵심 의도를 무시함.

# Output Format (JSON)
{"score": int, "reason": "string"}
"""

# ===========================================================
# 3. 핵심 함수 정의 (Core Functions)
# ===========================================================

def get_recommendation(row):
    """ [Step 1] GPT에게 추천 요청 (💰 API 사용) """
    user_msg = f"""
    Location: {row['Location']}
    Decibel: {row['Decibel']}
    Goal: {row['Goal']}
    Preference: {row['User Pref (Genre/Artist)']}
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4o", 
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT_DJ},
                {"role": "user", "content": user_msg}
            ],
            response_format={"type": "json_object"},
            temperature=0.7
        )
        return json.loads(response.choices[0].message.content)
    except Exception as e:
        print(f"❌ Gen Error: {e}")
        return {"spotify_search_params": {"search_queries": ["Error"]}}

def get_embedding_score(text1, text2):
    """ [Step 2] 코사인 유사도 계산 (🆓 내 컴퓨터 사용 / 무료) """
    try:
        # 내 컴퓨터 CPU로 벡터 변환
        embeddings = local_model.encode([text1, text2])
        # 유사도 계산
        similarity = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]
        return similarity
    except Exception as e:
        print(f"❌ Embed Error: {e}")
        return 0.0

def evaluate_result(row, generated_keywords, reasoning):
    """ [Step 3] LLM Judge에게 채점 요청 (💰 API 사용) """
    judge_msg = f"""
    [Input Context]
    Location: {row['Location']}, Goal: {row['Goal']}, Decibel: {row['Decibel']}
    User Pref: {row['User Pref (Genre/Artist)']}
    
    [Expected Vibe (Ground Truth)]
    {row['Expected Keywords / Vibe (기대 결과)']}
    
    [AI Generated Output]
    Keywords: {generated_keywords}
    Reasoning: {reasoning}
    
    위 내용을 바탕으로 점수(1~5)와 이유를 평가해 주세요.
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT_JUDGE},
                {"role": "user", "content": judge_msg}
            ],
            response_format={"type": "json_object"}
        )
        return json.loads(response.choices[0].message.content)
    except Exception as e:
        return {"score": 0, "reason": f"Error: {e}"}

# ===========================================================
# 4. 메인 실행 (Main Loop)
# ===========================================================
print("\n🚀 Soundscape 추천 품질 평가를 시작합니다...")
df = pd.read_csv(INPUT_FILE)
results = []

for index, row in df.iterrows():
    print(f"▶ Processing Case {index + 1}/{len(df)}: {row['Location']} ({row['Goal']})...")
    
    # [Step 1] 추천 생성
    gpt_output = get_recommendation(row)
    keywords_list = gpt_output.get('spotify_search_params', {}).get('search_queries', [])
    generated_text = ", ".join(keywords_list)
    reasoning = gpt_output.get('recommendation_meta', {}).get('reasoning', '')
    
    # [Step 2] 정량 평가 (임베딩 유사도 - 무료)
    expected_text = str(row['Expected Keywords / Vibe (기대 결과)'])
    embed_score = get_embedding_score(expected_text, generated_text)
    
    # [Step 3] 정성 평가 (LLM Judge - 유료)
    judge_result = evaluate_result(row, generated_text, reasoning)
    
    # 결과 저장
    results.append({
        "Case ID": row['ID'],
        "Scenario": f"{row['Location']} / {row['Goal']}",
        "Conflict Type": row['Type'], # 난이도 구분용
        "Expected": expected_text,
        "Generated": generated_text,
        "Reasoning": reasoning,
        "Embedding Score": round(embed_score, 3), # 0.0 ~ 1.0
        "LLM Score": judge_result['score'],       # 1 ~ 5
        "Judge Reason": judge_result['reason']
    })
    
    time.sleep(0.5) # API 속도 조절

# ===========================================================
# 5. 최종 결과 저장 및 요약
# ===========================================================
result_df = pd.DataFrame(results)
result_df.to_excel(OUTPUT_FILE, index=False)

avg_llm = result_df['LLM Score'].mean()
avg_embed = result_df['Embedding Score'].mean()
fatal_errors = len(result_df[result_df['LLM Score'] == 1])

print("\n" + "="*50)
print(f"✅ 평가 완료! 파일 저장됨: {OUTPUT_FILE}")
print(f" 평균 LLM 점수: {avg_llm:.2f} / 5.0 (목표: 4.5)")
print(f" 평균 임베딩 점수: {avg_embed:.2f} / 1.0")
print(f" 치명적 오류(1점): {fatal_errors}건 (목표: 0건)")
print("="*50)